In [ ]:
import numpy as np
import time
import csv
from dispensing import *
from scale_driver import *
from motor_driver import *

coms = {
    'scaleCom': 'COM7',
    'motorsCom': 'COM5',
}

motors = SerialConnection(coms['motorsCom'], 9600, 10)
dispenser_motor = Motor(motors, 0)

scale = Scale(coms['scaleCom'], 9600, 10)
scale.connect()
scale.tare()

In [ ]:
# Define your parameters
acceleration = 10
speed = 5
material = "SuperP"
version_inside = "Design2"
version_outside = "Design1"
motor = dispenser_motor
scale = scale
robot = robot

flush_material(dispenser_motor, 500)

In [ ]:
#Error test: Test with Super P and Design 2, see at which point the error does not change anymore significantly 
#Motor is doing 200steps for one full turn, Disgn2 has 25 pits which means 200/25 = 8 steps are neccesary to empty one pit

calibration = Calibration(acceleration, speed, material, version_inside, version_outside, motor, scale, robot)
glassware = Glassware()

#calibration steps = 200/n, 2x200/n, 3x200/n, 4x200/n, 5x200/n, 6x200/n, 7x200/n, 8x200/n, 9x200/n, 10x200/n, 11x200/n, 12x/200, 200
calibration_steps = [8, 16, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 200]
calibration_repeat = 10

calibration.calibrate(calibration_steps, calibration_repeat, "Vial1")

calibration.save_calibration()

In [ ]:

my_calibration = Calibration(100,100,"Test","NA", "NA")
my_calibration.calibrate([5, 30, 70], 2, dispenser_motor, scale, robot, "Vial1")
my_calibration.save_calibration()
(self, acceleration, speed, material, version_inside, version_outside, motor, scale, robot):